## User Trips Data Analysis

**Welcome to the Notebook for Thai Travel Thailand (TTT) User Trips Data Analysis**

This notebook will walk you through the process of extracting user trips from the preprocessed 'transaction.csv' dataset. 
But that's not all! Towards the conclusion of this notebook, we will delve into a series of intriguing questions related to the extracted user tripss.

### 📚 Import library

In [1]:
import pandas as pd

### 💻 Import data
We import both the **cleaned_user.csv** and **cleaned_transaction.csv** to pandas Dataframe.

In [2]:
cleaned_user_df = pd.read_csv('data/process/cleaned_user.csv')
cleaned_transaction_df = pd.read_csv('data/process/cleaned_transaction.csv')

In [3]:
cleaned_user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   1000 non-null   int64 
 1   hometown  1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [4]:
cleaned_transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41230 entries, 0 to 41229
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   datetime  41230 non-null  object
 1   user_id   41230 non-null  int64 
 2   province  41230 non-null  object
dtypes: int64(1), object(2)
memory usage: 966.5+ KB


### 🔨 Analysis

#### 💪🏻 Functions

First, we declear functions:
1. compare_string() for comparing user's hometown and province from cleaned_transaction DataFrame.
2. extract_trips() for extracing trip from each user.

In [5]:
def compare_string(str1, str2):
    # Convert both strings to lowercase and remove leading/trailing whitespaces
    str1 = str1.lower().strip()
    str2 = str2.lower().strip()
    # Compare the modified strings for equality
    return str1 == str2

def extract_trips(user_hometown, user_transaction):
    
    trips = []

    start_date = None
    end_date = None
    province_list = set() # Using set because we don't want duplication of province that user has visited
    
    for _, data in user_transaction.iterrows():
        province = data['province']

        # Check if we can begin or end the user's trip tracking
        if compare_string(province, user_hometown):
            if start_date is None:
                start_date = data['datetime']
            elif not province_list:
                # For cases of the data is consecutive hometown
                start_date = data['datetime']
            else:
                end_date = data['datetime']
        
        elif start_date is not None:
            # If begin tracking and not in the hometown, add the province to the province_list
            province_list.add(province)
        
        # Create trip row if there is a valid trip (Start from hometown, end at hometown)
        if end_date is not None and province_list:
        
            trip_info = {'start_date': start_date,
                      'end_date': end_date,
                      'province_list': province_list}
            
            trips.append(trip_info)
            
            start_date = end_date # Set end_date as start date for potential next trip
            end_date = None
            province_list = set()
    
    return trips

#### 🥽 Execution

Now we execute the function with all user_id

In [6]:
all_user_trips_df = pd.DataFrame()

# Get all user's id
user_ids = cleaned_user_df['user_id'].to_list()

for id in user_ids:

    # Get user's hometown
    hometown = cleaned_user_df.loc[cleaned_user_df['user_id'] == id, 'hometown'].iloc[0]

    # Extract user's transaction from cleaned_transaction_df
    user_transaction_df = cleaned_transaction_df[cleaned_transaction_df['user_id'] == id]
    user_transaction_df = user_transaction_df.drop(columns=['user_id'])
    user_transaction_df['datetime'] = user_transaction_df['datetime'].astype('datetime64[ns]') # Transform object to datetime type

    # Extract user's trips list
    user_trips = extract_trips(hometown, user_transaction_df)

    # Create user's trips DataFrame
    user_trips_df = pd.DataFrame(user_trips)
    user_trips_df.insert(0, 'user_id', id)

    # Aggregate user's trips DataFrame
    all_user_trips_df = pd.concat([all_user_trips_df, user_trips_df])

In [7]:
all_user_trips_df.head(10)

,user_id,start_date,end_date,province_list
0,1,2021-09-16 11:00:00,2021-09-16 23:00:00,{Sukhothai}
1,1,2021-09-16 23:00:00,2021-11-19 13:00:00,{Phayao}
2,1,2021-11-19 13:00:00,2021-12-20 12:00:00,"{Sukhothai, Chiang Rai, Lampang, Phayao}"
3,1,2021-12-20 12:00:00,2021-12-20 22:00:00,{Lampang}
4,1,2021-12-20 22:00:00,2021-12-25 22:00:00,"{Nonthaburi, Bangkok}"
5,1,2021-12-28 15:00:00,2021-12-29 08:00:00,{Lampang}
0,2,2021-01-02 11:00:00,2021-01-02 17:00:00,{Phra Nakhon Si Ayutthaya}
1,2,2021-04-05 10:00:00,2021-04-06 20:00:00,"{Ang Thong, Phra Nakhon Si Ayutthaya, Nonthaburi}"
2,2,2021-09-15 09:00:00,2021-09-16 06:00:00,{Nonthaburi}
3,2,2021-10-19 13:00:00,2021-10-20 00:00:00,"{Phra Nakhon Si Ayutthaya, Nonthaburi}"


Reset index and add trip_id column

In [8]:
result_df = all_user_trips_df.reset_index(drop=True)
result_df.insert(0, 'trip_id', range(1, len(result_df)+1))

### ✨ Result

In [9]:
display(result_df.info(), result_df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10341 entries, 0 to 10340
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   trip_id        10341 non-null  int64         
 1   user_id        10341 non-null  int64         
 2   start_date     10341 non-null  datetime64[ns]
 3   end_date       10341 non-null  datetime64[ns]
 4   province_list  10341 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 404.1+ KB


None

,trip_id,user_id,start_date,end_date,province_list
0,1,1,2021-09-16 11:00:00,2021-09-16 23:00:00,{Sukhothai}
1,2,1,2021-09-16 23:00:00,2021-11-19 13:00:00,{Phayao}
2,3,1,2021-11-19 13:00:00,2021-12-20 12:00:00,"{Sukhothai, Chiang Rai, Lampang, Phayao}"
3,4,1,2021-12-20 12:00:00,2021-12-20 22:00:00,{Lampang}
4,5,1,2021-12-20 22:00:00,2021-12-25 22:00:00,"{Nonthaburi, Bangkok}"
5,6,1,2021-12-28 15:00:00,2021-12-29 08:00:00,{Lampang}
6,7,2,2021-01-02 11:00:00,2021-01-02 17:00:00,{Phra Nakhon Si Ayutthaya}
7,8,2,2021-04-05 10:00:00,2021-04-06 20:00:00,"{Ang Thong, Phra Nakhon Si Ayutthaya, Nonthaburi}"
8,9,2,2021-09-15 09:00:00,2021-09-16 06:00:00,{Nonthaburi}
9,10,2,2021-10-19 13:00:00,2021-10-20 00:00:00,"{Phra Nakhon Si Ayutthaya, Nonthaburi}"


Now, the **result_df** is ready for answering questions. However, before proceeding, let's save our results to **output.csv** in the correct format.

In [10]:
def result_to_output(result_df):
    output_df = result_df.copy()
    output_df['start_date'] = output_df['start_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    output_df['end_date'] = output_df['end_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    output_df['province_list'] = output_df['province_list'].apply(lambda x: ', '.join(x))

    return output_df    

In [11]:
output_df = result_to_output(result_df)
output_df.to_csv('data/output/output.csv')

___

### 🧠 Question #1
What is the total number of trips?

We can get the answer from the len of the result DataFrame

In [12]:
print(f'The total number of trip(s): {len(result_df)}')

The total number of trip(s): 10341


___

### 🧠 Question #2
How many provinces are there in the trip with the most number of provinces?

In other words, we aim to find the maximum length of the sets in the 'province_list' column.

In [13]:
max_province = result_df['province_list'].apply(len).max()
print(f'There are {max_province} province(s) in the trip with the most number of provinces')

There are 12 province(s) in the trip with the most number of provinces


___

### 🧠 Question #3
What are the most common province pairs that people travel to in the same trip?


To answer this question, we need to find the common pair of all the province in province_list

In [14]:
# Select only province_list column
province_df = result_df['province_list']

In [15]:
display(province_df.info(), province_df.head())

<class 'pandas.core.series.Series'>
RangeIndex: 10341 entries, 0 to 10340
Series name: province_list
Non-Null Count  Dtype 
--------------  ----- 
10341 non-null  object
dtypes: object(1)
memory usage: 80.9+ KB


None

0                                 {Sukhothai}
1                                    {Phayao}
2    {Sukhothai, Chiang Rai, Lampang, Phayao}
3                                   {Lampang}
4                       {Nonthaburi, Bangkok}
Name: province_list, dtype: object

___

Exclude sets of provinces that are equal to 1, as we are looking for common pairs.

In [16]:
province_df = province_df[province_df.apply(lambda x: len(x) != 1)]

In [17]:
display(province_df.info(), province_df.head())

<class 'pandas.core.series.Series'>
Index: 5047 entries, 2 to 10339
Series name: province_list
Non-Null Count  Dtype 
--------------  ----- 
5047 non-null   object
dtypes: object(1)
memory usage: 78.9+ KB


None

2              {Sukhothai, Chiang Rai, Lampang, Phayao}
4                                 {Nonthaburi, Bangkok}
7     {Ang Thong, Phra Nakhon Si Ayutthaya, Nonthaburi}
9                {Phra Nakhon Si Ayutthaya, Nonthaburi}
10                                {Nonthaburi, Bangkok}
Name: province_list, dtype: object

___

Convert sets into lists and sort them to ensure consistent pairs. For example, we will have only ('Chiang Rai', 'Lampang') and not ('Lampang', 'Chiang Rai').

In [18]:
province_list = province_df.apply(lambda x: sorted(list(x))).to_list()

In [19]:
province_list[:5]

[['Chiang Rai', 'Lampang', 'Phayao', 'Sukhothai'],
 ['Bangkok', 'Nonthaburi'],
 ['Ang Thong', 'Nonthaburi', 'Phra Nakhon Si Ayutthaya'],
 ['Nonthaburi', 'Phra Nakhon Si Ayutthaya'],
 ['Bangkok', 'Nonthaburi']]

___

Next, we create all pair from each list in province_list

In [20]:
from itertools import combinations

province_pairs = []

for provinces in province_list:
  all_pairs = list(combinations(provinces, 2))
  province_pairs.extend(all_pairs)

In [21]:
print(f'There are {len(province_pairs)} pair(s)')

There are 21371 pair(s)


In [22]:
province_pairs[:5]

[('Chiang Rai', 'Lampang'),
 ('Chiang Rai', 'Phayao'),
 ('Chiang Rai', 'Sukhothai'),
 ('Lampang', 'Phayao'),
 ('Lampang', 'Sukhothai')]

___

Use province_pairs to create dataframe and calculate the most common pair

In [23]:
pair_df = pd.DataFrame(province_pairs, columns=['province1', 'province2'])

In [24]:
pair_df.head()

,province1,province2
0,Chiang Rai,Lampang
1,Chiang Rai,Phayao
2,Chiang Rai,Sukhothai
3,Lampang,Phayao
4,Lampang,Sukhothai


In [25]:
# Create new column for calculate the occurence of each pair
pair_df['occurrence(s)'] = 1

In [26]:
pair_df.head()

,province1,province2,occurrence(s)
0,Chiang Rai,Lampang,1
1,Chiang Rai,Phayao,1
2,Chiang Rai,Sukhothai,1
3,Lampang,Phayao,1
4,Lampang,Sukhothai,1


Compute common pairs by grouping with 'province 1' and 'province 2', then sum the occurrences of each unique pair.

In [27]:
common_pair_df = pair_df.groupby(['province1', 'province2']).sum().reset_index()

In [28]:
common_pair_df.sort_values(by=['occurrence(s)'], axis=0, ascending=False).head()

,province1,province2,occurrence(s)
347,Nonthaburi,Pathum Thani,464
369,Pathum Thani,Phra Nakhon Si Ayutthaya,415
323,Nakhon Pathom,Samut Sakhon,379
45,Bangkok,Pathum Thani,361
468,Ratchaburi,Samut Sakhon,354


The common province pair that people most travel in the same trip is **Nonthaburi <--> Pathum Thani** with the occurence of this pair at **464**

___

### 🎓 All Questions answer

1. **What are the most common province pairs that people travel to in the same trip?**
    - The total number of trip(s): 10341
2. **How many provinces are there in the trip with the most number of provinces?**
    - There are 12 province(s) in the trip with the most number of provinces
3. **What are the most common province pairs that people travel to in the same trip?**
    - The common province pair that people most travel in the same trip is Nonthaburi <--> Pathum Thani with the occurence of this pair at 464